In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df1=pd.read_csv("F:/Candidate_matching/data/cleaned_recommendation.csv")
df1.head()

,uniq_id,jobtitle,company,jobdescription,advertiserurl
0,90d755eb5b34547c31978ce7a6eb519f,full stack developer,Saicon Consultants Inc.,proficient in restful web servicesproficient i...,https://www.dice.com/jobs/detail/Full-Stack-De...
1,4b072aab5c96491c2d7f6731772b8825,full stack java engineer,RedRiver Systems L.L.C.,opportunity for an open / full stack java engi...,https://www.dice.com/jobs/detail/Full-Stack-Ja...
2,2e88654d255563f4d86baf821f92a8ac,full stack engineer,Dew Softech Inc,position: full stack engineerlocation...,https://www.dice.com/jobs/detail/Full-Stack-En...
3,39d56fc47d8fec2c2baa9e6e06142181,full-stack python developer,Cypress Group,my client is a world-class media and entertain...,https://www.dice.com/jobs/detail/Full%2526%252...
4,6ecd60ed9119b189293f55a7f5c5c88a,full stack developer,UST Global Inc,ust global is looking for full stack developer...,https://www.dice.com/jobs/detail/Full-Stack-De...


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 757 entries, 0 to 756
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   uniq_id         757 non-null    object
 1   jobtitle        757 non-null    object
 2   company         752 non-null    object
 3   jobdescription  757 non-null    object
 4   advertiserurl   757 non-null    object
dtypes: object(5)
memory usage: 29.7+ KB


In [5]:
df1=df1.dropna()

In [6]:
df1.shape

(752, 5)

In [7]:
df1['jobdescription'].head()

0    proficient in restful web servicesproficient i...
1    opportunity for an open / full stack java engi...
2    position:          full stack engineerlocation...
3    my client is a world-class media and entertain...
4    ust global is looking for full stack developer...
Name: jobdescription, dtype: object

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
tdif=TfidfVectorizer(stop_words='english')
df1['jobdescription']=df1['jobdescription'].fillna('')
tdif_matrix=tdif.fit_transform(df1['jobdescription'])
tdif_matrix.shape

(752, 14078)

In [10]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim=linear_kernel(tdif_matrix,tdif_matrix)
indices=pd.Series(df1.index, index=df1['jobtitle']).drop_duplicates()

In [11]:
def recommend(title):
    title = title.lower()
    if title not in df1['jobtitle'].str.lower().values:
        return f"The job title '{title}' does not exist in the dataset."

    # Get the index of the job title
    indx = df1[df1['jobtitle'].str.lower() == title].index[0]
    indx = df1.index.get_loc(indx)

    # Compute the distances and get recommendations
    distances = sorted(list(enumerate(cosine_sim[indx])), key=lambda x: x[1], reverse=True)[1:20]

    # Create a list to store job details
    jobs = []
    for i in distances:
        job_details = {
            'jobtitle': df1.iloc[i[0]].jobtitle,
            'uniq_id': df1.iloc[i[0]].uniq_id,
            'company': df1.iloc[i[0]].company,
            'jobdescription': df1.iloc[i[0]].jobdescription,
            'advertiserurl': df1.iloc[i[0]].advertiserurl
        }
        jobs.append(job_details)
    
    return jobs




In [12]:
recommend('Full Stack Developer')

[{'jobtitle': 'full stack java developer',
  'uniq_id': '1a84daa8f121b212e36750b6f64d09c8',
  'company': 'Kforce Inc.',
  'jobdescription': 'responsibilities:a kforce client in plano, texas (tx) is looking to hire full stack java developers.technology stack:* backend: java, spring, restful services using jersey/jax-rs* frontend: react js, angular js, bootstrap (at least one of these frameworks)essential functions:* an innovative and resourceful engineer who are passionate about creating high-quality, high performance and highly available software* love to develop highly scalable web applications and services that deliver delight to our customers* ready to roll up your sleeves and tackle technical problems and deliver products that delight customers* excited by working on a startup-like team, contributing to the vision and execution of cutting-edge products that solves problems and provides important benefits for our customers* display leadership qualities for other team members, includ